Import Libraries 

In [3]:
import os
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from random import random

Data Acquisition- Resistors 

In [4]:
#Get resistor data from https://octopart.com/electronic-parts/passive-components/resistors
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

#chrome_options = Options()
#chrome_options.add_argument("--headless")
#chrome_options.add_argument('--headless')
#chrome_options.add_argument('--disable-gpu')
#chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument("--start-maximized")
#chrome_options.add_argument("--window-size=1920x1080")
#,options=chrome_options

driver = webdriver.Chrome(executable_path='/Users/aarushi/Downloads/chromedriver')
page_count='310'
url ='https://octopart.com/search?category_id=4173&manufacturer_id=887&start='+page_count
#url = 'https://octopart.com/search?category_id=4173&start='+page_count
driver.get(url)
n=0
while n<10:
    try:
        ids = driver.find_elements_by_xpath('//div[@class="jsx-312275976 jsx-2649123136 mpn"]')[n].text
    except IndexError:
        print('detected')
        break
    print(ids)
    n+=1

MCMF0W4DF2490A50
MCSR08W1R00FTL
MCMR08X9763FTL
MC33162
MCMR08X000 PTL
MF12 20K
MCSR06X2702FTL
MC00625W040214M30
MC01W0805547K
MCPWR05FTEW7502


In [5]:
#Get capacitor data from https://octopart.com/electronic-parts/passive-components/resistors
driver = webdriver.Chrome(executable_path='/Users/aarushi/Downloads/chromedriver')
page_count='310'
url ='https://octopart.com/search?category_id=4166&start='+page_count
driver.get(url)
n=0
while n<10:
    try:
        ids = driver.find_elements_by_xpath('//div[@class="jsx-312275976 jsx-2649123136 mpn"]')[n].text
    except IndexError:
        print('detected')
        break
    print(ids)
    n+=1


EEUFR1C102L
C1210V104KCRACTU
T350E106K025AT
C0402C153K5RACTU
EEEFPJ101UAR
C0402C151J5GACTU
20VSK6
EEU-FR1E102
EEU-EB1H220S
C1206C560J5GACTU


Data Scrapping- Capacitors and Resistors

In [7]:
model_name_list=[]
capacitance = []
voltage = []
rating = []
tolerance = []
size = []
model_name_error=[]
resistance=[]
r_size=[]
restype=[]
power=[]
r_tolerance=[]
model_c=[]
model_r=[]

r_count=0
c_count=0
model_name=input("Enter part number:")
model_name=model_name.split()
n=len(model_name)
count=0
mm_count=0
while count < n:
    model=model_name[count]
    cap_count=0
    url = 'https://octopart.com/search?q='+model+'&currency=USD&specs=0'
    driver = webdriver.Chrome('/Users/aarushi/Downloads/chromedriver')
    driver.get(url)
    try:
        information = driver.find_elements_by_xpath('//div[@class="jsx-175549347 description"]')[0].text
    except IndexError:
        print("Detected by website for P/N:",model)
        n= model_name.index(model)
        model_name_error=model_name[0:n]
        model_name.clear()
        model_name=model_name_error
        break
    info_list=information.split()
    print(model,info_list)
    l=len(info_list)
    m=0
    if any("Cap" in word for word in info_list) or any("CAP" in word for word in info_list):
        model_c.append(model)
        while m<l:
            element=info_list[m]
            if mm_count==0 and element.isdecimal() and len(element)==4:
                try:
                    size[c_count]
                except IndexError:
                    size.append(element)
            elif 'mm' in element:
                mm_count+=1
                if mm_count==1:
                    if any(map(str.isdigit, element)):
                        try:
                            size[c_count]
                        except IndexError:
                            size.append(element)
                            if 'x' in info_list[m-1]or 'X' in info_list[m-1]:
                                newelement=info_list[m-2]+info_list[m-1]+size[-1]
                                size=size[:-1]
                                size.append(newelement)
                    else:
                        try:
                            size[c_count]
                        except IndexError:
                            size.append(info_list[m-1]+element)
                            if 'x' in info_list[m-2] or 'X' in info_list[m-2]:
                                newelement=info_list[m-3]+info_list[m-2]+size[-1]
                                size=size[:-1]
                                size.append(newelement)
                elif mm_count==2:
                    if any(map(str.isdigit, element)):
                        newelement=size[-1]+element
                        size = size[:-1]
                        size.append(newelement)
                        if 'x' in info_list[m-1]or 'X' in info_list[m-1]:
                            newelement=info_list[m-2]+info_list[m-1]+size[-1]
                            size=size[:-1]
                            size.append(newelement)
                    else:
                        newelement=size[-1]+info_list[m-1]+element
                        size = size[:-1]
                        size.append(newelement)
                        if 'x' in info_list[m-2]or 'X' in info_list[m-2]:
                            newelement=info_list[m-3]+info_list[m-2]+size[-1]
                            size=size[:-1]
                            size.append(newelement)
                else:
                    if any(map(str.isdigit, element)):
                        newelement=size[-1]+element
                        size = size[:-1]
                        size.append(newelement)
                    else:
                        newelement=element[-1]+info_list[m-1]+element
                        size = size[:-1]
                        size.append(newelement)
                        try:
                            size[c_count]
                        except IndexError:
                            size.append(info_list[m-1]+element)
                
            elif 'PF' in element or 'Pf' in element or 'pf' in element or 'pF' in element or 'nF' in element or 'NF' in element or 'Nf' in element or 'uF' in element or 'Uf' in element or 'uf' in element or 'UF' in element:
                cap_count+=1
                if cap_count==1:
                    if any(map(str.isdigit, element)):
                        try:
                            capacitance[c_count]
                        except IndexError:
                            capacitance.append(element)
                    else:
                        try:
                            capacitance[c_count]
                        except IndexError:
                            capacitance.append(info_list[m-1]+element)
                else:
                    try:
                        tolerance[c_count]
                    except IndexError: 
                        if any(map(str.isdigit, element)):
                            tolerance.append(element)
                        else:
                            tolerance.append(info_list[m-1]+element)
                
            elif 'Rtg' not in element and 'V' in element or 'volts' in element or 'Volts' in element or 'VDC' in element:
                if any(map(str.isdigit, element)):
                    try:
                        voltage[c_count]
                    except IndexError:
                        voltage.append(element)
                else:
                    try:
                        voltage[c_count]
                    except IndexError:
                        voltage.append(info_list[m-1]+element)
            elif '%' in element:
                try:
                    tolerance[c_count]
                except IndexError:
                    if any(map(str.isdigit, element)):
                        try:
                            tolerance[c_count]
                        except IndexError:
                            tolerance.append(element)
                    else:
                        try:
                            tolerance[c_count]
                        except IndexError:
                            tolerance.append(info_list[m-1]+element)
            m+=1
        try:
            capacitance[c_count]
        except IndexError:
            capacitance.append('NA')
        try:
            voltage[c_count]
        except IndexError:
            voltage.append('NA')
        try:
            size[c_count]
        except IndexError:  
            size.append('NA')
        try:
            tolerance[c_count]
        except IndexError:
            tolerance.append('NA')

        if capacitance != []:
            value = random()
            scaled_value = 1 + (value * (9 - 5))
            print(scaled_value)
            time.sleep(scaled_value)
        c_count+=1
        count+=1
        mm_count=0
    
    else:
        model_r.append(model)
        l=len(info_list)
        m=0
        while m<l:
            element=info_list[m]
            if 'Thick' in element or 'Thin' in element or 'Metal' in element:
                restype.append(element)
            #elif '402' in element or '603' in element or '805' in element or '1206' in element:
            elif element.isdecimal() and len(element)>=4:
                r_size.append(element)
            elif 'Ohm' in element or 'ohm' in element or 'Ω' in element or 'OHM' in element:
                if any(map(str.isdigit, element)):
                    try:
                        resistance[r_count]
                    except IndexError:
                        resistance.append(element)
                else:
                    try:
                        resistance[r_count]
                    except IndexError:
                        resistance.append(info_list[m-1]+ element)
            elif 'Wire' not in element and 'W' in element or 'Mw' in element or 'MW' in element:
                if any(map(str.isdigit, element)):
                    try:
                        power[r_count]
                    except IndexError:
                        power.append(element)
                else:
                    try:
                        power[r_count]
                    except IndexError:
                        power.append(info_list[m-1]+element)
            elif '%' in element:
                    if any(map(str.isdigit, element)):
                        r_tolerance.append(element)
                    else:
                        r_tolerance.append(info_list[m-1]+element)
            m+=1
        try:
            resistance[r_count]
        except IndexError:
            resistance.append('NA')
        try:
            restype[r_count]
        except IndexError:
            restype.append('NA')
        try:
            r_size[r_count]
        except IndexError: 
            r_size.append('NA')
        try:
            r_tolerance[r_count]
        except IndexError:
            r_tolerance.append('NA')
        try:
            power[r_count]
        except IndexError:
            power.append('NA')

        if resistance != []:
            value = random()
            scaled_value = 1 + (value * (9 - 5))
            print(scaled_value)
            time.sleep(scaled_value)
        r_count+=1
        count+=1 
        


print('Capacitors:')
print('P/N:',len(model_c),'Tolerance:', len(tolerance),'Voltage:',len(voltage),'Capacitance:',len(capacitance),'Size:',len(size))
error_finder=[len(model_c),len(tolerance),len(voltage),len(capacitance),len(size)]
if error_finder[0]==error_finder[1]:
    if error_finder[1]==error_finder[2]:
        if error_finder[2]==error_finder[3]:
            if error_finder[3]==error_finder[4]:
                print("No error")
            elif error_finder[2]>error_finder[3]:
                print(capacitance)
            else:
                print(size)
        elif error_finder[2]>error_finder[3]:
            print(voltage)
        else:
            print(capacitance)
    elif error_finder[1]>error_finder[2]:
        print(tolerance)
    else:
        print(voltage)
elif error_finder[0]>error_finder[1]:
    print(model_c)
    
else:
    print(tolerance)
dfc = pd.DataFrame(
    {'model_c': model_c,
     'capacitance': capacitance,
     'voltage':voltage,
     'size':size,
     'tolerance':tolerance
    }
)
if dfc.empty==False:
    dfc['capacitance']=dfc['capacitance'].str.replace(";","")
    dfc['capacitance']=dfc['capacitance'].str.replace(",","")
    dfc['voltage']=dfc['voltage'].str.replace(",","")
    dfc['tolerance']=dfc['tolerance'].str.replace(",","")
    dfc['size']=dfc['size'].str.replace(";","")
    dfc['size']=dfc['size'].str.replace("(","")
    dfc['size']=dfc['size'].str.replace(")","")
    dfc['voltage']=dfc['voltage'].str.replace(";","")
print(dfc)

print(len(model_r),len(r_tolerance),len(resistance),len(power),len(r_size))
print('Resistors:')
dfr = pd.DataFrame(
    {'model_r': model_r,
     'type':restype,
     'resistance': resistance,
     'power':power,
     'size':r_size,
     'tolerance':r_tolerance
    }
)
print('P/N:',len(model_r),'Type:', len(restype),'Resistance:',len(resistance),'Power:',len(power),'Tolerance:',len(r_tolerance), 'Size:',len(r_size))
error_finder_r=[len(model_r),len(restype),len(resistance),len(power),len(r_size),len(r_tolerance)]
if error_finder_r[0]==error_finder_r[1]:
    if error_finder_r[1]==error_finder_r[2]:
        if error_finder_r[2]==error_finder_r[3]:
            if error_finder_r[3]==error_finder_r[4]:
                if error_finder_r[4]==error_finder_r[5]:
                    print("No error")
                elif error_finder_r[4]>error_finder_r[4]:
                    print(r_size)
                else: 
                    print(r_tolerance)
            elif error_finder_r[3]>error_finder_r[4]:
                print(power)
            else:
                print(r_size)
        elif error_finder_r[2]>error_finder_r[3]:
            print(resistance)
        else:
            print(power)
    elif error_finder_r[1]>error_finder_r[2]:
        print(restype)
    else:
        print(resistance)
elif error_finder_r[0]>error_finder_r[1]:
    print(model_r)
else:
    print(restype)
    
if dfr.empty==False:
    dfr['resistance']=dfr['resistance'].str.replace("Ohm"," Ω")
    dfr['resistance']=dfr['resistance'].str.replace(",","")
    dfr['power']=dfr['power'].str.replace(",","")
    dfr['tolerance']=dfr['tolerance'].str.replace(",","")
    dfr['size']=dfr['size'].str.replace(",","")

print(dfr)

Enter part number:C1210V104KCRACTU MCMF0W4DF2490A50 MCSR08W1R00FTL MCMR08X9763FTL MC33162 T350E106K025AT C0402C153K5RACTU EEEFPJ101UAR
C1210V104KCRACTU ['1210', 'Arcshield', 'HV', '100nF', 'Ceramic', 'Multilayer', 'Cap;', '500VDC;', '+125C;', 'X7R', 'Dielec;', '+/-10%']
1.6626671691053692
MCMF0W4DF2490A50 ['Through', 'Hole', 'Metal', 'Film', 'Resistor,', 'Mcmf0W4', 'Series,', '249', 'Ohm,', '250', 'Mw,', '-', '0.5%,', '250', 'V,', 'Axial', 'Leaded', 'Rohs', 'Compliant:', 'Yes']
3.675970535772867
MCSR08W1R00FTL ['Res', 'Anti-Sulfured', 'Thick', 'Film', '0805', '1', 'Ohm', '1%', '1/8W', '-200ppm/°C', 'to', '400ppm/°C', 'Ceramic', 'Paper', 'T/R']
4.8873856437878995
MCMR08X9763FTL ['Res', 'General', 'Purpose', 'Thick', 'Film', '0805', '976k', 'Ohm', '1%', '1/8W', '±100ppm/°C', 'Ceramic', 'Paper', 'T/R']
3.184905958310321
MC33162 ['PTC', 'Resettable', 'Fuse', '100A', '10.3s', '30VDC', '0.02', 'Ohm', '2-Pin', 'Radial', 'Leaded']
4.733051493647974
T350E106K025AT ['Capacitor', 'Tantalum', 'Sol